In [1]:
#pip install yfinance
#pip install panel
#pip install hvplot

In [2]:
from pathlib import Path
import pandas as pd
import yfinance as yf
import numpy as np
import panel as pn
pn.extension()
import hvplot.pandas
from watermark import watermark
%matplotlib inline

In [3]:
#System Requirements

print(watermark())
print(watermark(iversions=True, globals_=globals()))


Last updated: 2022-10-18T18:51:38.898030-04:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.31.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD
CPU cores   : 16
Architecture: 64bit

numpy   : 1.21.5
yfinance: 0.1.77
hvplot  : 0.8.0
pandas  : 1.3.5
panel   : 0.13.1



## Data pull

In [4]:
# manual inputs, to be replaced by stock and portfolio dfs and variables from analytics team

tickr = "NIO"
tickrs = ["META","AMZN","GOOG","AAPL","NFLX"]
mkttickr = "^GSPC"

#variable assignment
stock = yf.Ticker(tickr)
portfolio = yf.Tickers(tickrs)
mkt = yf.Ticker(mkttickr)
ticker_names = [tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],"SP500"]

In [5]:
#historical data dataframes

stock_history_df = stock.history(period = "5y")
portfolio_history_df = portfolio.history(period = "5y")
mkt_history_df = mkt.history(period = "5y")

# #preview
# portfolio_history_df.tail()

[*********************100%***********************]  5 of 5 completed


## Df setup

In [6]:
#prices df
prices_df = pd.concat([
     stock_history_df["Close"],portfolio_history_df["Close"],mkt_history_df["Close"]],
     axis = 'columns',
     join = 'inner'
    )

#renaming columns, plotting prices (no SP500)
prices_df.columns = [ticker_names[0],ticker_names[1],ticker_names[2],ticker_names[3],ticker_names[4],ticker_names[5],ticker_names[6]]

In [7]:
# combined returns dataframe (close price, daily, 5 years)

#combine data frames using only Close column
returns_df = pd.concat([
     stock_history_df["Close"],portfolio_history_df["Close"],mkt_history_df["Close"]],
     axis = 'columns',
     join = 'inner'
)
#renaming columns
returns_df.columns = [ticker_names[0],ticker_names[1],ticker_names[2],ticker_names[3],ticker_names[4],ticker_names[5],ticker_names[6]]

#calculate returns
returns_df = returns_df.pct_change().dropna()

In [8]:
# Cummulative returns
cumulative_returns_df = (1 + returns_df).cumprod()

## Calculations

In [9]:
#correlation
correlation = returns_df.corr()
# correlation

In [10]:
#sharpe ratio calculation
sharpe_ratios = (returns_df.mean() * 252) / (returns_df.std() * np.sqrt(252))
sharpe_ratios = sharpe_ratios.sort_values()
sharpe_ratios = sharpe_ratios.to_frame()
sharpe_ratios.columns = ["Sharpe"]
# sharpe_ratios

In [11]:
#sortino ratios
sortino_ratios = (returns_df.mean() * 252) / (returns_df[returns_df<0].std() * np.sqrt(252))
sortino_ratios = sortino_ratios.sort_values()
sortino_ratios = sortino_ratios.to_frame()
sortino_ratios.columns = ["Sortino"]
# sortino_ratios

In [12]:
#volatility 
volatility = returns_df.std() * np.sqrt(252)
volatility = volatility.sort_values()
volatility = volatility.to_frame()
volatility.columns = ["Volatility"]

In [13]:
# Stock Covariance Calculation
covariance_df = returns_df.cov()
covariance_df = covariance_df[["SP500"]]
covariance_df.columns = ["Covariance"]
# display(covariance_df)

#beta calculation
variance = returns_df['SP500'].var()
# print(variance)

beta_df = returns_df.cov()/variance
beta_df = beta_df[["SP500"]]
beta_df.columns = ["Beta"]
# beta_df

## Plotting

### Closing Prices 

In [14]:
# tickers_wd = pn.widgets.CheckButtonGroup(options= list(prices_df.columns), value = [prices_df.columns[0]], name='Ticker')

# def pricesplot_df(tickers_wd):
#     df = pd.DataFrame(prices_df[tickers_wd])
#     return df

# prices_plot = hvplot.bind(pricesplot_df, tickers_wd).interactive(width=600).hvplot(
#     title = "Historical Prices - 5 years",
#     ylabel = "Stock Price")
# prices_plot


In [15]:
prices_plot = prices_df.drop(columns = "SP500").hvplot.line(
    title = "Historical Prices - 5 years",
    ylabel = "Stock Price"
    )
prices_plot 

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

### Daily Returns 

In [16]:
# ireturns = returns_df.interactive()
# returns_df.hvplot.bar()

## Cummulative Returns 

In [17]:
cum_returns_plot = cumulative_returns_df.hvplot(
    title = "Historical Cummulative Returns - 5 years",
    ylabel = "Return %"
    )
cum_returns_plot 

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [18]:
# tickers_wd = pn.widgets.Select(options= [tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],mkttickr], name='Ticker')

# def returnsplot_df(tickers_wd):
#     df = pd.DataFrame(cumulative_returns_df[tickers_wd])
#     return df

# returns_plot = hvplot.bind(returnsplot_df,tickers_wd).interactive(width=600).hvplot(
#     title = "Historical Cummulative Returns - 5 years",
#     ylabel = "Return %"
#     )

# returns_plot

In [19]:
# def pricesplot_df(tickers_wd):
#     df = pd.DataFrame(prices_df[tickers_wd])
#     return df

# pricesplot = hvplot.bind(pricesplot_df,tickers_wd).interactive(width=600).hvplot(
#     title = "Historical Prices - 5 years",
#     ylabel = "Stock Price"
#     )

# pricesplot 

## Ratios table

In [20]:
#Create df starting by correlation
ratios_df = returns_df.corr()
ratios_df.drop(ratios_df.columns[0:-1],axis=1,inplace = True) #drop columns for correlation against each other, leave only correlation against SP500
ratios_df.rename(columns = {"SP500":"Correlation"},inplace = True) #rename column from SP500 to Correlation
ratios_df["Volatility"] = returns_df.std() * np.sqrt(252)
ratios_df["Sharpe"] =  (returns_df.mean() * 252) / (returns_df.std() * np.sqrt(252))
ratios_df["Sortino"] = (returns_df.mean() * 252) / (returns_df[returns_df<0].std() * np.sqrt(252))
ratios_df["Covariance"] = covariance_df["Covariance"]
ratios_df["Beta"] = beta_df["Beta"]
ratios_df


,Correlation,Volatility,Sharpe,Sortino,Covariance,Beta
NIO,0.299199,0.998086,0.609487,1.212228,0.000271,1.307775
META,0.807446,0.345279,0.875615,1.194839,0.000253,1.220926
AMZN,0.671721,0.363878,0.287054,0.402971,0.000221,1.070408
GOOG,0.796716,0.315096,0.588452,0.807715,0.000227,1.099391
AAPL,0.669721,0.418293,0.097077,0.124398,0.000254,1.226818
NFLX,0.507947,0.480961,0.033642,0.041319,0.000221,1.069875
SP500,1.000000,0.228347,0.385196,0.460321,0.000207,1.000000


In [21]:
# Make interactive ratios bar plot

ratio_wd = pn.widgets.Select(options= list(ratios_df.columns), name='Ratio')

def ratiosplot_df(ratio_wd):
    df = pd.DataFrame(ratios_df[ratio_wd])
    return df

ratios_bar = hvplot.bind(ratiosplot_df, ratio_wd).interactive(width=600).hvplot.bar(
    title = "Key Ratios",
    xlabel = "Ticker"
    )
ratios_bar

AttributeError: 'function' object has no attribute 'bar'

## Panel

In [ ]:
#DASHBOARD

#Create widgets
tickers_wd = pn.widgets.Select(options= [tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],mkttickr], name='Ticker', value= tickrs[0])
df_widget = pn.widgets.DataFrame(ratios_df, name='Key Ratios')

#Define functions and variables for an interactive sidebar 
def company_name(ticker):
    name = pn.pane.Markdown("# " + yf.Ticker(ticker).info['longName'])
    return name

def company_desc(ticker):
    desc = pn.pane.Markdown(yf.Ticker(ticker).info['longBusinessSummary'])
    return desc
    
def company_website(ticker):
    website = pn.pane.Markdown(yf.Ticker(ticker).info['website'])
    return website

def company_logo(ticker):
    logo = yf.Ticker(ticker).info['logo_url']
    return logo

company_name_sidebar = pn.bind(company_name,tickers_wd)
company_desc_sidebar = pn.bind(company_desc,tickers_wd)
company_website_sidebar = pn.bind(company_website,tickers_wd)
company_logo_sidebar = pn.bind(company_logo,tickers_wd)

# Create dashboard using FastListTemplate from Panel Library

template = pn.template.FastListTemplate(
    title = "Stock Analysis",
    sidebar = [
        tickers_wd,
        company_name_sidebar,
        company_website_sidebar,
        company_desc_sidebar],
    main=[
        pn.Row(pn.Column(prices_plot, margin=(0,25)), 
                 cum_returns_plot), 
        pn.Row(pn.Column(df_widget, margin=(0,25)), 
                 ratios_bar)
               ],
    theme="dark"
)

#Show template
template.servable();      
template.show()

In [ ]:
# # FIRST TRY

# #widgets
# tickers_wd = pn.widgets.Select(options= [tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],mkttickr], name='Ticker')
# df_widget = pn.widgets.DataFrame(ratios_df, name='Key Ratios')

# #companyinfo - TBD on how to use widget to automatically update
# company_name = yf.Ticker(tickers_wd.value).info['longName']
# companydesc = yf.Ticker(tickers_wd.value).info['longBusinessSummary']
# companywebsite = yf.Ticker(tickers_wd.value).info['website']
# logourl = yf.Ticker(tickers_wd.value).info['logo_url']

# #template
# template = pn.template.FastListTemplate(
#     title = company_name + ' - Stock Analysis',
#     sidebar = [
#         tickers_wd.servable(area="sidebar"), #HAVING ISSUES TRYING TO USE TICKERS WIDGET TO AUTOMATICALLY UPDATE COMPANY NAME, DESC, AND WEBSITE WHEN USED.
#         pn.pane.Markdown("# " + company_name),
#         pn.pane.Markdown(companydesc)],
#         # pn.bind(tickers,company_name,companydesc,companywebsite)],             HAVING ISSUES TRYING TO USE TICKERS WIDGET TO AUTOMATICALLY UPDATE COMPANY NAME, DESC, AND WEBSITE WHEN USED.
#     sidebar_footer = companywebsite,
#     main=[
#         # tickers_wd,
#         pn.Row(pn.Column(prices_plot, margin=(0,25)), 
#                  cum_returns_plot), 
#         pn.Row(pn.Column(df_widget, margin=(0,25)), 
#                  ratios_bar)
#                ],
#     theme="dark",
#     logo = logourl
# )

# template.servable();      
# template.show()